# 🚀 Smart Retail AI - Training on Google Colab

**Train model với GPU miễn phí - 9 bước đơn giản!**

## 📋 Checklist trước khi bắt đầu:
- ✅ Runtime → Change runtime type → **GPU (T4)**
- ✅ Có file **kaggle.json** (lấy tại: https://www.kaggle.com/settings/account)

## ⏱️ Thời gian ước tính:
- Setup: 5 phút
- Download datasets: 10-15 phút  
- Training: 45-60 phút
- **Total: ~1 giờ 20 phút**

## 📊 Kết quả mong đợi:
- Gender Accuracy: > 90%
- Emotion Accuracy: > 75%
- Age MAE: < 4.0 years

---

**🚀 Chạy từng cell theo thứ tự từ trên xuống!**

In [ ]:
# ============================================================
# CELL 1: CHECK GPU
# ============================================================
import torch

print("=" * 60)
print("GPU CHECK")
print("=" * 60)

print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n[OK] GPU T4 is ready!")
else:
    print("\n[ERROR] NO GPU! Please enable:")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU")
    
print("=" * 60)

✅ Đã cài đặt xong các thư viện!


In [ ]:
# This cell is removed - use Cell 1 instead

PyTorch version: 2.9.0+cu126


AttributeError: module 'torch.cuda' has no attribute 'is_is_available'

In [ ]:
# ============================================================
# CELL 2: CLONE REPOSITORY
# ============================================================
import os

print("=" * 60)
print("CLONE REPOSITORY")
print("=" * 60)

repo_dir = '/content/repo'

if not os.path.exists(repo_dir):
    print("\n[INFO] Cloning from GitHub...")
    !git clone https://github.com/khoiabc2020/age-gender-emotion-detection.git {repo_dir}
    print("[OK] Repository cloned!")
else:
    print("\n[INFO] Repository exists, pulling latest...")
    !cd {repo_dir} && git pull
    print("[OK] Repository updated!")

# Navigate to training directory
%cd {repo_dir}/training_experiments

print(f"\n[INFO] Current directory:")
!pwd

print("\n[INFO] Directory structure:")
!ls -la

print("=" * 60)

In [ ]:
# ============================================================
# CELL 3: SETUP KAGGLE API
# ============================================================
from google.colab import files
import os
import shutil

print("=" * 60)
print("SETUP KAGGLE API")
print("=" * 60)

print("\n[INFO] Please upload your kaggle.json file")
print("   Get it from: https://www.kaggle.com/settings/account")
print("   → Scroll to 'API' section")
print("   → Click 'Create New API Token'")
print("   → Download kaggle.json\n")

# Upload kaggle.json
uploaded = files.upload()

if 'kaggle.json' in uploaded:
    # Setup Kaggle directory
    kaggle_dir = '/root/.kaggle'
    os.makedirs(kaggle_dir, exist_ok=True)
    
    # Move and set permissions
    kaggle_json = os.path.join(kaggle_dir, 'kaggle.json')
    shutil.move('kaggle.json', kaggle_json)
    os.chmod(kaggle_json, 600)
    
    print("\n[OK] Kaggle API configured successfully!")
    print("=" * 60)
else:
    print("\n[ERROR] kaggle.json not found!")
    print("   Please upload kaggle.json to continue")
    print("=" * 60)
    raise Exception("Kaggle API key required")

# ============================================================
# OPTION 2: Download từ Google Drive (file zip)
# ============================================================
if not USE_GITHUB:
    # Đường dẫn file zip trên Drive
    import glob
    zip_files = glob.glob('/content/drive/MyDrive/Colab_Training/training_experiments_*.zip')

    if zip_files:
        # Lấy file mới nhất
        latest_zip = max(zip_files, key=os.path.getctime)
        print(f"📦 Tìm thấy file: {os.path.basename(latest_zip)}")

        # Giải nén
        print("📂 Đang giải nén...")
        with zipfile.ZipFile(latest_zip, 'r') as zip_ref:
            zip_ref.extractall('/content/project')
        print("✅ Đã giải nén code")
    else:
        print("⚠️  Không tìm thấy file zip trên Drive")
        print("   Hãy upload file zip vào: /content/drive/MyDrive/Colab_Training/")

# Kiểm tra cấu trúc
project_dir = Path('/content/project/training_experiments')
if project_dir.exists():
    print(f"\n✅ Code đã sẵn sàng tại: {project_dir}")
else:
    print("\n⚠️  Cần kiểm tra lại cấu trúc thư mục")

In [ ]:
# ============================================================
# CELL 4: DOWNLOAD DATASETS FROM KAGGLE
# ============================================================

print("=" * 60)
print("DOWNLOADING DATASETS")
print("=" * 60)

# Install kagglehub if needed
%pip install -q kagglehub

import kagglehub
from pathlib import Path

print("\n[INFO] Downloading FER2013 (Emotion Recognition Dataset)...")
print("   Size: ~50MB, Time: ~5-10 minutes\n")
fer2013_path = kagglehub.dataset_download("msambare/fer2013")
print(f"[OK] FER2013 downloaded to: {fer2013_path}")

print("\n[INFO] Downloading UTKFace (Age & Gender Dataset)...")
print("   Size: ~300MB, Time: ~10-15 minutes\n")
utkface_path = kagglehub.dataset_download("jangedoo/utkface-new")
print(f"[OK] UTKFace downloaded to: {utkface_path}")

# Save paths for later use
paths_file = '/content/dataset_paths.txt'
with open(paths_file, 'w') as f:
    f.write(f"FER2013: {fer2013_path}\n")
    f.write(f"UTKFace: {utkface_path}\n")

print(f"\n[OK] All datasets downloaded successfully!")
print(f"[INFO] Paths saved to: {paths_file}")
print("=" * 60)

In [ ]:
# ============================================================
# CELL 5: INSTALL DEPENDENCIES
# ============================================================

print("=" * 60)
print("INSTALLING DEPENDENCIES")
print("=" * 60)

print("\n[INFO] Installing required packages...")
%pip install -q albumentations tensorboard onnx onnxscript onnxruntime

print("\n[OK] All dependencies installed!")
print("=" * 60)

In [ ]:
# ============================================================
# CELL 6: START TRAINING
# ============================================================

import os
from pathlib import Path

print("=" * 60)
print("STARTING TRAINING")
print("=" * 60)

# Read dataset paths
with open('/content/dataset_paths.txt') as f:
    lines = f.read().strip().split('\n')
    fer2013_path = lines[0].split(': ')[1]
    utkface_path = lines[1].split(': ')[1]

# Training configuration
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001
SAVE_DIR = "/content/checkpoints"

print(f"\n[CONFIG] Training Configuration:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   Dataset (Emotion): {fer2013_path}")
print(f"   Dataset (Age/Gender): {utkface_path}")
print(f"   Save Directory: {SAVE_DIR}")
print(f"   Device: GPU T4")

# Pull latest code (in case of updates)
print(f"\n[INFO] Pulling latest code...")
%cd /content/repo
!git pull
print("[OK] Code updated!")

# Navigate to training directory
%cd /content/repo/training_experiments

# Verify training script exists
if not os.path.exists('train_colab_simple.py'):
    print("\n[WARN] Training script not found!")
    print("   Trying to re-clone repository...")
    %cd /content
    !rm -rf /content/repo
    !git clone https://github.com/khoiabc2020/age-gender-emotion-detection.git /content/repo
    %cd /content/repo/training_experiments

print(f"\n[START] Starting training... (This will take ~45-60 minutes)")
print("=" * 60)

# Run training with simplified script
!python train_colab_simple.py \
    --data_dir {fer2013_path} \
    --epochs {EPOCHS} \
    --batch_size {BATCH_SIZE} \
    --lr {LEARNING_RATE} \
    --save_dir {SAVE_DIR}

print("\n" + "=" * 60)
print("[OK] TRAINING COMPLETED!")
print("=" * 60)

In [ ]:
# ============================================================
# CELL 8: SAVE RESULTS TO GOOGLE DRIVE
# ============================================================

import shutil
from pathlib import Path
from datetime import datetime

print("=" * 60)
print("SAVING TO GOOGLE DRIVE")
print("=" * 60)

# Source and destination
checkpoint_dir = Path("/content/checkpoints")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
drive_dir = Path(f"/content/drive/MyDrive/SmartRetailAI_Models/training_{timestamp}")
drive_dir.mkdir(parents=True, exist_ok=True)

if checkpoint_dir.exists():
    print(f"\n📦 Copying files to Google Drive...")
    
    file_count = 0
    total_size = 0
    
    for file in checkpoint_dir.glob("*"):
        if file.is_file():
            shutil.copy(file, drive_dir / file.name)
            size_mb = file.stat().st_size / 1024 / 1024
            print(f"   ✅ {file.name} ({size_mb:.1f} MB)")
            file_count += 1
            total_size += size_mb
    
    print(f"\n✅ Successfully saved {file_count} files ({total_size:.1f} MB total)")
    print(f"📁 Location: {drive_dir}")
else:
    print("\n❌ Checkpoint directory not found!")
    print("   Training may have failed. Check errors above.")

print("=" * 60)

In [ ]:
# ============================================================
# CELL 9: TRAINING SUMMARY
# ============================================================

import json
from pathlib import Path

print("=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)

# Try to load results
results_file = Path("/content/checkpoints/training_results.json")

if results_file.exists():
    with open(results_file) as f:
        results = json.load(f)
    
    print(f"\n📊 Training Results:")
    print(f"   Gender Accuracy: {results.get('gender_accuracy', 'N/A')}%")
    print(f"   Emotion Accuracy: {results.get('emotion_accuracy', 'N/A')}%")
    print(f"   Age MAE: {results.get('age_mae', 'N/A')} years")
    print(f"   Training Time: {results.get('training_time', 'N/A')} minutes")
    print(f"   Best Epoch: {results.get('best_epoch', 'N/A')}")
else:
    print(f"\n📊 Training completed!")
    print(f"   Check your files in Google Drive:")
    print(f"   MyDrive → SmartRetailAI_Models")

print(f"\n📥 Next Steps:")
print(f"   1. Download model.onnx from Google Drive")
print(f"   2. Copy to your edge app: ai_edge_app/models/")
print(f"   3. Test the edge app with new model")
print(f"   4. Update documentation with results")

print("\n✅ ALL DONE!")
print("=" * 60)

In [ ]:
import shutil
from pathlib import Path

# Option 1: Copy từ Google Drive (nếu đã upload)
DRIVE_DATA_DIR = '/content/drive/MyDrive/age_gender_emotion_data'
LOCAL_DATA_DIR = Path('/content/project/training_experiments/data/processed')

if os.path.exists(DRIVE_DATA_DIR):
    print(f"📁 Copy dữ liệu từ Drive...")
    shutil.copytree(DRIVE_DATA_DIR, LOCAL_DATA_DIR, dirs_exist_ok=True)
    print(f"✅ Đã copy dữ liệu")
else:
    print(f"⚠️  Chưa có dữ liệu trên Drive")
    print(f"   Upload dữ liệu vào: {DRIVE_DATA_DIR}")
    print(f"   Hoặc sử dụng cell tiếp theo để download từ Kaggle")

In [ ]:
from pathlib import Path

data_dir = Path('/content/project/training_experiments/data/processed')
train_dir = data_dir / 'train'
val_dir = data_dir / 'val'
test_dir = data_dir / 'test'

print("📊 Kiểm tra dữ liệu:")
print(f"   Train: {train_dir.exists()}")
print(f"   Val: {val_dir.exists()}")
print(f"   Test: {test_dir.exists()}")

if train_dir.exists():
    train_images = list(train_dir.glob('**/*.jpg')) + list(train_dir.glob('**/*.png'))
    val_images = list(val_dir.glob('**/*.jpg')) + list(val_dir.glob('**/*.png')) if val_dir.exists() else []
    test_images = list(test_dir.glob('**/*.jpg')) + list(test_dir.glob('**/*.png')) if test_dir.exists() else []

    print(f"\n📸 Số lượng ảnh:")
    print(f"   Train: {len(train_images)}")
    print(f"   Val: {len(val_images)}")
    print(f"   Test: {len(test_images)}")

    if len(train_images) == 0:
        print("\n⚠️  Chưa có ảnh trong thư mục train!")
else:
    print("\n⚠️  Chưa có dữ liệu processed!")

In [ ]:
import sys
import os
from pathlib import Path

# Thêm project vào Python path
project_dir = Path('/content/project/training_experiments')
sys.path.insert(0, str(project_dir))
os.chdir(project_dir)

# Cấu hình training
EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
USE_QAT = True
USE_DISTILLATION = True

print("🚀 Bắt đầu training tự động...")
print("=" * 60)
print(f"⚙️  Cấu hình:")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   QAT: {USE_QAT}")
print(f"   Distillation: {USE_DISTILLATION}")
print("=" * 60)

# Build command
cmd = f"python train_week2_lightweight.py --data_dir data/processed --epochs {EPOCHS} --batch_size {BATCH_SIZE} --lr {LEARNING_RATE} --save_dir checkpoints/week2_colab --num_workers 2"

if USE_QAT:
    cmd += " --use_qat"
if USE_DISTILLATION:
    cmd += " --use_distillation"

# Chạy training
os.system(cmd)

print("\n" + "=" * 60)
print("✅ Training hoàn tất!")

In [ ]:
import shutil
from datetime import datetime
from pathlib import Path

# Tạo thư mục lưu kết quả với timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_dir = Path(DRIVE_RESULTS_DIR) / f'training_{timestamp}'
results_dir.mkdir(parents=True, exist_ok=True)

# Copy checkpoints
checkpoint_dir = Path('/content/project/training_experiments/checkpoints/week2_colab')
if checkpoint_dir.exists():
    print(f"📦 Copy checkpoints...")
    shutil.copytree(checkpoint_dir, results_dir / 'checkpoints', dirs_exist_ok=True)
    print(f"✅ Đã copy checkpoints")

# Copy logs
logs_dir = checkpoint_dir / 'logs'
if logs_dir.exists():
    print(f"📊 Copy logs...")
    shutil.copytree(logs_dir, results_dir / 'logs', dirs_exist_ok=True)
    print(f"✅ Đã copy logs")

# Copy ONNX model
onnx_file = checkpoint_dir / 'mobileone_multitask.onnx'
if onnx_file.exists():
    print(f"📄 Copy ONNX model...")
    shutil.copy2(onnx_file, results_dir / 'mobileone_multitask.onnx')
    print(f"✅ Đã copy ONNX model")

print(f"\n✅ Tất cả kết quả đã được lưu tại:")
print(f"   {results_dir}")